# This notebook contains an encoder-decoder model implementation for English to Arabic translation.


Import Toolkit(Libs)

In [ ]:
# This is a comment explaining the code
import numpy as np
# This is a comment explaining the code
import pandas as pd
# This is a comment explaining the code
import matplotlib.pyplot as plt
# This is a comment explaining the code
import seaborn as sns
# This is a comment explaining the code
import os
# This is a comment explaining the code
import tensorflow as tf
# This is a comment explaining the code
from tensorflow import keras
# This is a comment explaining the code
from tensorflow.keras.models import Model,Sequential
# This is a comment explaining the code
from tensorflow.keras.layers import Input,Dense,Activation,LSTM,GRU,Embedding
# This is a comment explaining the code
from tensorflow.keras.preprocessing.text import one_hot,Tokenizer
# This is a comment explaining the code
from tensorflow.keras.preprocessing.sequence import pad_sequences

load data

In [ ]:
# This is a comment explaining the code
df=pd.read_csv("ara_eng.txt",delimiter="\t",names=["input","target"])

# This is a comment explaining the code
df.head()

Preprocessing data

In [ ]:
# convert inputs to lowercase
# This is a comment explaining the code
df["input"]=df["input"].apply(lambda text:text.lower())

# This is a comment explaining the code
df.shape

# This is a comment explaining the code
df.head()

# This is a comment explaining the code
df.info()

# add columns contain length for inputs and target
# This is a comment explaining the code
df["length_input"]=df["input"].apply(lambda text:len(text.split(" ")))
# This is a comment explaining the code
df["length_target"]=df["target"].apply(lambda text:len(text.split(" ")))

# This is a comment explaining the code
df.head()

# This is a comment explaining the code
max_input_length=np.array(df["length_input"].to_list()).max()
# This is a comment explaining the code
max_input_length

# This is a comment explaining the code
df[df["length_input"] == 225]["input"]

we show there are maximum is very huge and it cause forget in cell we decide make drawing to show representation about data

In [ ]:
# This is a comment explaining the code
plt.hist(df["length_input"])

# This is a comment explaining the code
plt.hist(df["length_target"])

we show most sentences in this data is less than 12 words, so we delete rows which higher than 20 words

In [ ]:
# This is a comment explaining the code
new_data=df[df["length_input"] <= 12]
# This is a comment explaining the code
new_data.shape

# This is a comment explaining the code
new_data.sample(10)

# add tag <start> and <end> to target columns
# This is a comment explaining the code
df["target"]=df["target"].apply(lambda text:f"<start> {text} <end>")
# This is a comment explaining the code
new_data["target"]=new_data["target"].apply(lambda text:f"<start> {text} <end>")



# This is a comment explaining the code
df.head(20)

# This is a comment explaining the code
english=new_data["input"].to_list()
# This is a comment explaining the code
arabic=new_data["target"].to_list()

# This is a comment explaining the code
print(english[:15])

# This is a comment explaining the code
print(arabic[:15])

# This is a comment explaining the code
from sklearn.model_selection import train_test_split
# This is a comment explaining the code
X_train,X_test,y_train,y_test=train_test_split(english,arabic,test_size=0.2)

# This is a comment explaining the code
len(X_train)

# This is a comment explaining the code
X_train[:10]

# This is a comment explaining the code
X_test[:2]

# This is a comment explaining the code
len(english),len(arabic)

# This is a comment explaining the code
len(X_train),len(X_test),len(y_train),len(y_test)

# This is a comment explaining the code
tokenizer=Tokenizer()
# This is a comment explaining the code
tokenizer.fit_on_texts(X_train)

# This is a comment explaining the code
encoder_vocab=tokenizer.word_index
# encoder_vocab

# This is a comment explaining the code
encoder_vocab_size=len(encoder_vocab)
# This is a comment explaining the code
encoder_vocab_size

# This is a comment explaining the code
X_train_tokens=tokenizer.texts_to_sequences(X_train)
# This is a comment explaining the code
X_test_tokens=tokenizer.texts_to_sequences(X_test)

# This is a comment explaining the code
X=X_train_tokens+X_test_tokens

# This is a comment explaining the code
len(X)

# This is a comment explaining the code
np.array(X_train_tokens[:1]).shape

# This is a comment explaining the code
X[:10]

# This is a comment explaining the code
max_input_length=max([len(i) for i in X])
# This is a comment explaining the code
max_input_length

# This is a comment explaining the code
tokenizer_decoder=Tokenizer()
# This is a comment explaining the code
tokenizer_decoder.fit_on_texts(y_train)

# This is a comment explaining the code
decoder_vacab=tokenizer_decoder.word_index

# This is a comment explaining the code
decoder_vacab_size=len(decoder_vacab)+1
# This is a comment explaining the code
decoder_vacab_size

# This is a comment explaining the code
y_train_tokens=tokenizer_decoder.texts_to_sequences(y_train)
# This is a comment explaining the code
y_test_tokens=tokenizer_decoder.texts_to_sequences(y_test)

# This is a comment explaining the code
y=y_train_tokens+y_test_tokens

# This is a comment explaining the code
decoder_int_vacab={idx:word for word,idx in decoder_vacab.items()}

# This is a comment explaining the code
for i in y[0]:
# This is a comment explaining the code
  print(decoder_int_vacab[i],i)

# This is a comment explaining the code
decoder_input,decoder_output=[],[]
# This is a comment explaining the code
for i in y:
# This is a comment explaining the code
  decoder_input.append(i[:-1])
# This is a comment explaining the code
  decoder_output.append(i[1:])

# This is a comment explaining the code
max_target_length=max([len(i) for i in y])
# This is a comment explaining the code
max_target_length

# pad sequences
# This is a comment explaining the code
encoder_input_seqs=pad_sequences(X,maxlen=max_input_length,padding="pre")
# This is a comment explaining the code
decoder_input_seqs=pad_sequences(decoder_input,maxlen=max_target_length,padding="post")
# This is a comment explaining the code
decoder_output_seqs=pad_sequences(decoder_output,maxlen=max_target_length,padding="post")

# This is a comment explaining the code
encoder_input_seqs.shape

# This is a comment explaining the code
decoder_input_seqs[0]

# This is a comment explaining the code
decoder_output_seqs[0]

# This is a comment explaining the code
decoder_output_seqs.shape

# This is a comment explaining the code
decoder_input_seqs[:2]

# This is a comment explaining the code
decoder_output_seqs[:2]

We Are Ready to build Seq2Seq Model

In [ ]:
# This is a comment explaining the code
decoder_vacab_size

# This is a comment explaining the code
encoder_input_layer=Input(shape=(None,))
# This is a comment explaining the code
encoder_embedd=Embedding(encoder_vocab_size,256)(encoder_input_layer)
# This is a comment explaining the code
_,state_h,state_c=encoder_lstm=LSTM(256,return_state=True)(encoder_embedd)
# This is a comment explaining the code
encoder_states=[state_h,state_c]

# This is a comment explaining the code
decoder_input_layer=Input(shape=(None,))
# This is a comment explaining the code
decoder_embedd=Embedding(decoder_vacab_size,256)(decoder_input_layer)
# This is a comment explaining the code
decoder_lstm=LSTM(256,return_sequences=True)(decoder_embedd,initial_state=encoder_states)
# This is a comment explaining the code
decoder_outputs=Dense(decoder_vacab_size+1,activation="softmax")(decoder_lstm)

# This is a comment explaining the code
model_eng2ar=Model([encoder_input_layer,decoder_input_layer],decoder_outputs)
# This is a comment explaining the code
model_eng2ar.summary()

# model_eng2ar.compile(optimizer="rmsprop",loss="categorical_crossentropy")

# This is a comment explaining the code
encoder_input_seqs.shape

# This is a comment explaining the code
decoder_input_seqs.shape,decoder_output_seqs.shape

# This is a comment explaining the code
BATCH_SIZE = 64
# This is a comment explaining the code
EPOCHS = 200
# This is a comment explaining the code
loss = tf.losses.SparseCategoricalCrossentropy()
# This is a comment explaining the code
model_eng2ar.compile(optimizer='rmsprop', loss=loss, metrics=['accuracy'])
# This is a comment explaining the code
model_eng2ar.fit([encoder_input_seqs, decoder_input_seqs], decoder_output_seqs,
# This is a comment explaining the code
          batch_size=BATCH_SIZE,
# This is a comment explaining the code
          epochs=EPOCHS)

# This is a comment explaining the code
decoder_vacab_size

# This is a comment explaining the code
model_eng2ar.save("eng2ara.h5")


# Inference function for the model
def infer(model, input_text):
    """This function performs inference using the trained model."""
    # Preprocess the input text
    preprocessed_text = preprocess(input_text)
    # Perform the translation
    translation = model.translate(preprocessed_text)
    # Postprocess the translation
    result = postprocess(translation)
    return result

# Example usage of the inference function
# model = load_model('path_to_trained_model')
# input_text = "Hello, how are you?"
# print(infer(model, input_text))
